# fz-brent: Standalone Example (without fzd)

This notebook demonstrates how to use the Brent root-finding algorithm **without** coupling it to a model via `fzd`.

This is useful for:
- Testing the Brent algorithm implementation
- Understanding the algorithm interface for root finding
- Running the algorithm against a simple Python test function

**Requirements:** R and `rpy2` must be installed.

## Setup: Install Dependencies

In [ ]:
# Install fz framework and rpy2
%pip install -q git+https://github.com/Funz/fz.git
%pip install -q rpy2

## Example 1: Load the R Algorithm via rpy2

Load the Brent algorithm by sourcing the `.R` file and creating an instance of the S3 class.

In [ ]:
from rpy2 import robjects

# Source the R algorithm file
robjects.r.source(".fz/algorithms/brent.R")
r_globals = robjects.globalenv

# Create an instance with custom options
r_algo = robjects.r["Brent"](ytarget=0.0, ytol=0.01, xtol=0.01, max_iterations=100)
print(f"Algorithm class: {robjects.r['class'](r_algo)[0]}")
print(f"Algorithm options: ytarget=0.0, ytol=0.01, xtol=0.01")

## Example 2: Define a Test Function

Brent's method finds the root of a 1D function. We define `cos(π·x)` whose root in `[0, 1]` is at `x = 0.5`.

In [ ]:
import math

def test_function(x):
    """cos(pi * x) — root at x = 0.5 in [0, 1]"""
    return math.cos(math.pi * x)

# Define input variable range (1D only for Brent)
input_vars = {"x": (0.0, 1.0)}
output_vars = ["y"]

print(f"Test function: cos(π·x)")
print(f"Input variables: {input_vars}")
print(f"Known root: x = 0.5 (where cos(π·0.5) = {test_function(0.5)})")

## Example 3: Run the Algorithm Lifecycle

Execute the full root-finding lifecycle:
1. `get_initial_design()` — get 3 initial bracketing points from R
2. Evaluate the test function at those points (in Python)
3. `get_next_design()` — iteratively refine until convergence (or `list()` when done)
4. `get_analysis()` — get final root-finding analysis from R

In [ ]:
# Step 1: Get initial design from R
r_input_vars = robjects.r('list(x = c(0.0, 1.0))')
r_output_vars = robjects.StrVector(["y"])

r_design = r_globals['get_initial_design'](r_algo, r_input_vars, r_output_vars)

# Convert R design to Python
design = []
for i in range(len(r_design)):
    point = {}
    r_point = r_design[i]
    for name in r_point.names:
        point[name] = r_point.rx2(name)[0]
    design.append(point)

print(f"Initial design: {len(design)} points")
for i, point in enumerate(design):
    print(f"  Point {i}: x={point['x']:.4f}, f(x)={test_function(point['x']):.6f}")

In [ ]:
# Step 2: Evaluate the test function at initial points
all_X = list(design)
all_Y = [test_function(p["x"]) for p in design]

print(f"Evaluated {len(all_Y)} points")
for i, (p, y) in enumerate(zip(all_X, all_Y)):
    print(f"  x={p['x']:.4f} → f(x)={y:.6f}")

In [ ]:
# Step 3: Iterative loop (get_next_design until list() is returned)
iteration = 0
while True:
    # Get intermediate analysis
    try:
        r_all_Y = robjects.FloatVector(all_Y)
        r_tmp = r_globals['get_analysis_tmp'](r_algo, r_design, r_all_Y)
        if r_tmp is not None and r_tmp.names is not None and 'text' in list(r_tmp.names):
            print(r_tmp.rx2('text')[0])
    except Exception:
        pass

    # Get next design
    r_next = r_globals['get_next_design'](r_algo, r_design, r_all_Y)

    if len(r_next) == 0:
        print(f"\nAlgorithm converged after {iteration} iteration(s)")
        break

    # Convert next design to Python
    next_points = []
    for i in range(len(r_next)):
        point = {}
        r_point = r_next[i]
        for name in r_point.names:
            point[name] = r_point.rx2(name)[0]
        next_points.append(point)

    # Evaluate and accumulate
    new_Y = [test_function(p["x"]) for p in next_points]
    all_X.extend(next_points)
    all_Y.extend(new_Y)

    # Update R design for next iteration
    for i in range(len(r_next)):
        r_design = robjects.r('c')(r_design, robjects.r('list')(r_next[i]))

    iteration += 1

In [ ]:
# Step 4: Get final analysis from R
r_analysis = r_globals['get_analysis'](r_algo, r_design, robjects.FloatVector(all_Y))

print("=" * 60)
print("ANALYSIS RESULTS")
print("=" * 60)

if r_analysis.names is not None and 'text' in list(r_analysis.names):
    print(r_analysis.rx2('text')[0])

if r_analysis.names is not None and 'data' in list(r_analysis.names):
    r_data = r_analysis.rx2('data')
    print("Data:")
    for name in r_data.names:
        val = r_data.rx2(name)
        try:
            print(f"  {name}: {val[0]}")
        except Exception:
            print(f"  {name}: {val}")

## Example 4: Visualize Root Finding

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Extract x values and outputs
x_vals = [p['x'] for p in all_X]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Function curve with evaluated points and root
ax = axes[0]
xi = np.linspace(0.0, 1.0, 200)
yi = np.cos(np.pi * xi)
ax.plot(xi, yi, 'b-', linewidth=2, label='cos(π·x)')
ax.axhline(y=0, color='grey', linestyle='--', linewidth=1, label='Target (y=0)')
ax.scatter(x_vals, all_Y, c=range(len(x_vals)), cmap='autumn', edgecolors='black',
           s=60, linewidth=0.5, zorder=5, label='Evaluated points')

# Mark the root
r_data = r_analysis.rx2('data')
root_x = r_data.rx2('root')[0]
root_y = r_data.rx2('value')[0]
ax.plot(root_x, root_y, 'r*', markersize=15, label=f'Root found (x={root_x:.4f})', zorder=10)
ax.axvline(x=0.5, color='green', linestyle=':', linewidth=1, alpha=0.7, label='True root (x=0.5)')

ax.set_xlabel('x')
ax.set_ylabel('f(x) = cos(π·x)')
ax.set_title('Brent Root Finding - cos(π·x) = 0')
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

# Plot 2: Convergence of root approximation
ax = axes[1]
# Every 3 points is one iteration (Brent returns triplets: a, b, c)
# The second point in each triplet (b) is the best root approximation
b_values = [x_vals[i] for i in range(1, len(x_vals), 3)]
ax.plot(range(len(b_values)), b_values, 'o-', color='steelblue', markersize=6)
ax.axhline(y=0.5, color='green', linestyle='--', label='True root (x=0.5)')
ax.set_xlabel('Iteration')
ax.set_ylabel('Root approximation (b)')
ax.set_title('Convergence of Root Approximation')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Alternative: Load via fz plugin system

You can also load R algorithms using fz's `load_algorithm()`, which handles the rpy2 wrapping automatically:

In [ ]:
from fz.algorithms import load_algorithm

# Load R algorithm via fz (auto-wraps with rpy2)
algo = load_algorithm(".fz/algorithms/brent.R", ytarget=0.0, ytol=0.01, xtol=0.01)

# The wrapper provides the same Python interface
design = algo.get_initial_design({"x": (0.0, 1.0)}, ["y"])
print(f"Got {len(design)} points from R algorithm via fz wrapper")
for i, p in enumerate(design):
    print(f"  Point {i}: {p}")